In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import logistic

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Out of sample prediction
아까 까지는 in smaple prediction

In [7]:
file1="c:/temp/credit_LN4.csv"
credit=pd.read_csv(file1)

x=credit['balance']
y=credit['default']
y=pd.get_dummies(y)
y=y['Yes']
x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=0)

In [ ]:
#Logistic regression with train set
#Prediction with test set (3000개에 대해서 CCR을 계산한다.)



# sen, spec, ccr 이런거 정의 시험에 아주 잘 나옴. 1-spe:false positive rate
#true positive rate: 진양성: 
#fpr : 위양성

In [11]:
file2="c:/temp/smarket.csv"
smarket=pd.read_csv(file2)
smarket

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...,...
1245,2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
1246,2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
1247,2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up
1248,2005,0.130,-0.955,0.043,0.422,0.252,1.42236,-0.298,Down


In [13]:
#Lag n: 그 주식 n일 전에 주가 상승률, Volume: 거래량, 

#이런 데이터를 뭐라고 하지? : 시계열 데이터 time series data
#이 데이터에서 시간변수: year
#v파이썬에서 시계열 불러들이려면 시간변수가 index 칼럼이 되어야함.
#parse_date=True : index col을 날짜로 인식해라

file2="c:/temp/smarket.csv"
smarket=pd.read_csv(file2, index_col='Year', parse_dates=True)
smarket


,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...
2005-01-01,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
2005-01-01,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
2005-01-01,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up


In [14]:
smarket.index=smarket.index.year
smarket
#1월 1일 데이터가 아니라 년 데이터인데 2001로 표시하게 하는 거

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
2001,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2001,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
2001,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
2001,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...
2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up


In [19]:
#2001-2004년 을 train, 2005년 test
#왜? 이런 시계열 데이터는 전처럼 무작위로 나눠서 하면 의미가 없음. 이런 주가..는 과거 데이터 이용해서 미래 시점 예측하는게 잘 맞는지 봐야하는 거임. 
# 어떻게 나눠?

#train_set=smarket[smarket.index<=2004]
#test_set=smarket[smarket.index>=2005]


train_set=smarket.loc[:2004]
test_set=smarket.loc[2005:]
train_set

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
...,...,...,...,...,...,...,...,...
2004,0.046,0.342,0.904,0.038,-0.749,0.9561,-0.431,Down
2004,-0.431,0.046,0.342,0.904,0.038,0.9220,0.715,Up
2004,0.715,-0.431,0.046,0.342,0.904,0.9830,-0.007,Down


In [20]:
#나머지 실습은 lec4 보고 해볼것!

# Lec 5: Classification using K-NN

supervised learning :관심있는 y 변수를 예측하는 것 
근데 y의 종류가 두가지 있음. Classification 범주형 변수/Regression 연속형 변수

Classification 하는 거 중에 저번에 Logistic 배웠고 오늘은 K-NN 배운다.

K-NN : 예측대상의 주위에 누가 있는지 보는것 nearest neighborhood 

장점
단점
적절한 k 선택의 어려움
slow classification phase

2:2면 각각 거리 계산해서 평균 거리 짧은 쪽으로 판정함.

유클리드형 distance
2차원보다 클때도 피타고라스로 계산



In [25]:
# tomato (Sweet=6, crucnch=4), green been (3,7)

np.sqrt(((6-3)**2)+((4-7)**2))


4.242640687119285

In [26]:
#k=1 인 경우 : 가장 가까운 애 하나 선택 -> 토마토는 과일
#k=3 인 경우: 과과프 -> 과일

## 2.2 Choosing an appropriate k

-underfitting? overfitting? : k가 너무 크면 노이즈가 많아져서, 가까이 있는 패턴을 무시하는 위험 있음. 분산 감소: k가 클수록, x라는 애의 판단은 바뀌진 않음. 분산은 variation, k가 클수록 y를 판단하는 데에서 분산이 바뀌진 않음, k가 작으면 variability가 굉장히 심해서 값이 잘 바뀜.

k small->분산 증가
k large->분산 감소: 값의 판단에 있어서 그 결과가 잘 바뀌지 않는다. 

- k=전체 x가 어디에있든 간에 항상 결과 같음. 
- k=1이면 좋은친구가 100명있어도 제일 가까이ㅣㅆ는게 나쁜친구면 나쁜애로 판명됨.

large k: 오류를 감수한 일반화 가능
small k: 일반화 어렵

### K를 정하자

k=3

k=4

k=5

이렇게 정하고, test set으로 예측 성과 판단 해서 제일 예측 잘하는 걸로..

관례적으로 k는 훈련데이터 갯수의 제곱근으로 시작.


거리, distance 계산하기위해선 x변수가 필요함.


###  표준화

표준화하고나면 측정단위에 의존하지 않음.
표준화ㅏ고나면 -3에서 3사이의 값으로 나타남.

(x-평균)/표준분포

In [27]:
x=[10,15,20,70,90]
x

[10, 15, 20, 70, 90]

In [33]:
x_st=(x-np.mean(x))/np.sqrt(np.var(x))
x_st #표준화된 값

array([-0.95036598, -0.79708114, -0.64379631,  0.88905204,  1.50219138])

In [35]:
x_stan=(x-np.mean(x))/np.std(x)
x_stan

array([-0.95036598, -0.79708114, -0.64379631,  0.88905204,  1.50219138])

### 최소-최대 정규화 min-max normalization

In [36]:
np.max(x)

90

In [37]:
np.min(x)

10

In [39]:
(x-np.min(x))/(np.max(x)-np.min(x))

array([0.    , 0.0625, 0.125 , 0.75  , 1.    ])

In [41]:
#이런 표준화, 정규화 거쳐야 측정단위에 의존하지 않고 거리 측정할 수 있음

#표준 정규화: -3,3을 약간 벗어나기도 함. 그래서 음수와 양수의 한정되지 않은 범위에 놓여있다는 것. 
#최소최대: 항상 [0,1] 의 범위 가짐

